<h1 style="color:blue">Praktikum 10. Keelemudelid</h1>
<h4>15. november 2016 </h4>
<h4>Ülesannete esitamise tähtaeg 29. november.</h4>

### Ülesanne 1. Sõnasoovitaja (6p)

Juba mitu aastakümmet on mobiilsetes seadmetes sõnumite saatjaid muserdanud automaatne
sõnade soovitamine. Käesolevas praktikumis kirjutame ise sarnaselt toimiva süsteemi, mida
saab konfigureerida, et see paremini või halvemini töötaks.

Lahenduseks kasutame järgmist lähenemist:
* treenime mingil eestikeelsel tekstil keele n-grammi baasmudeli, et hoida soovitused keeleliselt nii korrektsed kui võimalik ka puhkudel, kui kasutaja tekstis pole vastavaid
n-gramme veel nähtud,
* kasutaja eelistuste leidmiseks treenime *online* stiilis kasutaja keelemudeli, mida hakkame vastavalt kasutaja sisendile täiendama,
* ühendame baasmudeli ja kasutajamudeli üheks süsteemiks,
* loeme kasutaja sisendit, kuni kasutaja annab märku, et soovib sõna ennustamisel abi.

#### A. Treenime baasmudeli ja salvestame kettale
* treenime mingil hulgal piibli kaustas olevatel raamatutel unigrammi, bigrammi ja trigrammi alammudelid
    * mida rohkem raamatuid kaasame, seda parem tuleb mudel, kuid ka seda mälunõudlikum ning aeglasem
    * n-grammi mudeli talletamiseks on tarvis sagedustabelit, mille
         * veergudeks on (n-1)-grammid
             * unigrammi korral on j-ndas reas üks veerg, mis sisaldab j-nda tekstisõne sagedust dokumendis
         * ridadeks tekstisõned (sõnad + olulisemad kirjavahemärgid)
         * (i,j) väärtuseks n-grammi sagedus dokumendis
             * n-gramm moodustunud veeru (n-1)-grammist + rea unigrammist
* mudeleid treenides ei tohi unustada metasõnavara
     * [unk] *out-of-vocabulary* lahendamiseks
         * [unk] treenimiseks võib iga sõna esimesel esinemisel suurendada vastaval kontekstil ((n-1)-grammil) [unk] sagedust
     * (n-1) [lause algus] enne igat lauset
     * (n-1) [lause lõpp] pärast igat lauset
     * lisaks võib agregeerida omaette
         * numbrid
         * harvemad kirjavahemärgid
         * nimed
* silume mudelid (valida üks kahest meetodist)
     * üldine Laplace’i silumine (liidame kõigile sagedustele mingi fikseeritud arvu vahemikust [0,1])
     * Good-Turing sageduse kalibreerimine (vt loenguslaidid)
* jagame tekkinud sagedustabelite elementide väärtused vastavate tabelite veergude summaga
     * nt bigrammi mudeli j-ndas veerus oleva elemendi jagame läbi bigrammi mudeli j-nda veeru summaga
         * teoorias vastab C(n-gramm) / C((n-1)-gramm)
* salvestame kettale tekkinud mudelid ja nende ridade ning veergude kujutused n-grammidest indeksiteks


**Lahendusena esitada kood, mis täidab ülesande 1.A nõudeid. (3p)**

#### B. Initsialiseerime kasutaja mudeli
* loome baasmudelile identsete tunnustega unigrammi, bigrammi ja trigrammi sagedustabelid
     * kui baasmudelis kirjeldab 5. rida sõna “koer” järgnevust erinevatele unigrammidele, siis kaunistab ka antud bigrammi mingit rida (miks mitte 5ndat?) sõna “koer” ja tema järgnevuste sagedusi erinevatele unigrammidele
     * tarvis lugeda sisse baasmudeli alammudelite ridade ja veergude kujutused n-grammidest indeksiteks
* silume üldise Laplace’iga
     * hästi väike arv [0,1]
* salvestame kasutaja mudeli ja ridade ning veergude kujutused kettale

**Lahendusena esitada skript, mis täidab ülesande 1.B nõudeid. (1p)**

#### C. Paneme süsteemi kokku
* loeme sisse baasmudeli tabelid ja sõnaraamatud
* loeme sisse kasutaja mudeli tabelid ja sõnaraamatud
* pärime kasutajalt lõputult sisendtekste
     * talletame kasutaja alammudelitesse kasutaja sisendi
         * kui kasutaja sisestas “”Täna on taevas päike.”, siis näiteks
             * suurendame trigrammi sagedustabelis ühe võrra väärtust kohal (“taevas”,”Täna on”)
             * suurendame bigrammi sagedustabelis ühe võrra väärtust kohal (“taevas”,”on”)
             * suurendame unigrammi sagedustabelis ühe võrra “taevas” sagedust
         * kui kasutaja ei lõpetanud teksti lauset lõpetava kirjavahemärgiga
             * jätame lõppu (n-1) [lause lõpp] lisamata
     * kui kasutaja ei lõpetanud teksti lauset lõpetava kirjavahemärgiga
         * prindime soovitatava sõna vastavalt tõenäosusele, kus suvalise sõna tõenäosus avaldub valemiga

$P(w_i|w_{i-2}w_{i-1}) = \alpha _1(\beta_1P_{bm}(w_i|w_{i-2}w_{i-1})+\beta_2P_{bm}(w_i|w_{i-1}+\beta_3P_{bm}(w_i))+\alpha_2(\beta_1P_{um}(w_i|w_{i-2}w_{i-1})+\beta_2P_{um}(w_i|w_{i-1})+\beta_3P_{um}(w_i)) $

   * $ \alpha_i $ ja $\beta_j$ on parameetrid, mille puhul
       * $sum(\alpha_i) = 1$
       * $sum(\beta_j) = 1$
   * $P_{bm}$ on baasmudeli vastavad alammudelid
   * $P_{um}$ on kasutaja mudeli vastavad alammudelid
       * $P_{um} $ tõenäosuseid ei saanud ette arvutada, kuna teadmine süsteemist muutub iga kasutaja sisendiga
           * peame vastavad sagedused veeru summadega igal arvutusel jooksutama
           * saame terve veeru korraga arvutada
           
* kui kasutaja tahab skripti sulgeda, salvestame enne uuendatud kasutaja mudeli kettale           
       
       

**Lahendusena esitage skript, mis vastab ülesande 1.C nõuetele. (2p)**